In [ ]:
import os
import sys
import glob
import pickle
from time import strftime, localtime

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_transform as tft

if '..' not in sys.path:
    sys.path.append('..')
from deep_utils import *

%matplotlib inline

First, let's fix the seed of the RNG, for reproducibility purposes

In [ ]:
with open('/dev/random', 'rb') as fid:
    seed = int.from_bytes(fid.read(4), 'little')
tf.random.set_seed(seed)
print('Seed: {}'.format(seed))

#### Load the data

In [ ]:
generator_IDs = [1, 2]
n_generators = len(generator_IDs)
data_folders = [f'../data/var_H_G{gen_id}/IEEE14_D=2_DZA=60.0/' for gen_id in generator_IDs]
var_names = [lbl for gen_id in generator_IDs for lbl in (f'omega_G{gen_id}',f'Pe_G{gen_id}')]
n_vars = len(var_names)
inertia = {key: np.arange(2,11) + i/3 for i,key in enumerate(('training', 'test', 'validation'))}
time, x, y = load_data(data_folders, generator_IDs, inertia, var_names, max_block_size = 300)
x['train'] = x.pop('training')
y['train'] = y.pop('training')
N_vars, N_training_traces, N_samples = x['train'].shape

#### Normalize the data

In [ ]:
x_train_mean = np.mean(x['train'], axis=(1,2))
x_train_std = np.std(x['train'], axis=(1,2))
for key in x:
    x[key] = tf.constant([(x[key][i].numpy() - m) / s for i,(m,s) in enumerate(zip(x_train_mean, x_train_std))])

Let's take a look at the data:

In [ ]:
H_values = np.array([2,10])

fig = plt.figure(figsize=(8, 2 * N_vars))
gs = fig.add_gridspec(N_vars, 3)
ax = []
for i in range(N_vars):
    ax.append(fig.add_subplot(gs[i,:2]))
    ax.append(fig.add_subplot(gs[i,2]))

idx_min, _ = np.where(y['train'] == H_values.min())
idx_max, _ = np.where(y['train'] == H_values.max())

xlim_distr = None
if xlim_distr is not None:
    lim = xlim_distr
else:
    lim = 0

for i in range(N_vars):
    j = i * 2
    ax[j].plot(time, x['train'].numpy()[i, idx_min[:5], :].T, 'k', lw=1)
    ax[j].plot(time, x['train'].numpy()[i, idx_max[:5], :].T, 'r', lw=1)

    n,edges = np.histogram(np.ndarray.flatten(x['train'][i,:,:].numpy()[idx_min,:]), \
                           bins=100, range=[-4,4], density=True)
    lim = n.max() if n.max() > lim else lim
    ax[j + 1].plot(n, edges[:-1], 'k', linewidth=1.2, label=f'H={H_values[0]:g}')
    n,edges = np.histogram(np.ndarray.flatten(x['train'][i,:,:].numpy()[idx_max,:]), \
                           bins=100, range=[-4,4], density=True)
    lim = n.max() if n.max() > lim else lim
    ax[j + 1].plot(n, edges[:-1], 'r', linewidth=1.2, label=f'H={H_values[1]:g}')
    ax[j + 1].set_yticklabels([])

for a in ax:
    for side in 'right', 'top':
        a.spines[side].set_visible(False)

for i in range(N_vars):
    j = i * 2
    ax[j + 1].plot([0, lim*1.05], [0,0], '--', lw=1, color=[.6,.6,.6])
    ax[j].get_shared_x_axes().join(ax[0], ax[j])
    ax[j+1].get_shared_x_axes().join(ax[1], ax[j+1])
    ax[j+1].get_shared_y_axes().join(ax[j], ax[j+1])
    ax[j].set_ylabel(var_names[i].replace('_',' '))
ax[(N_vars - 1) * 2].set_xlabel('Time [s]')
ax[(N_vars - 1) * 2 + 1].set_xlabel('Fraction')
ax[1].legend(loc='upper right')
fig.tight_layout()

### Build the network
The basic network topology used here is taken from the following paper:

George, D., & Huerta, E. A. (2018). Deep neural networks to enable real-time multimessenger astrophysics. Physical Review D, 97(4), 044039. http://doi.org/10.1103/PhysRevD.97.044039

In [ ]:
def make_preprocessing_pipeline_1D(N_samples, N_units, kernel_size, activation_fun, activation_loc, input_name):
    if activation_fun is not None:
        if activation_fun.lower() not in ('relu',):
            raise Exception(f'Unknown activation function {activation_fun}')
        if activation_loc is None:
            raise Exception(f'Must specify activation function location')
        elif activation_loc.lower() not in ('after_conv', 'after_pooling'):
            raise Exception('activation_loc must be one of "after_conv" or "after_pooling"')
    inp = keras.Input(shape=(N_samples, 1), name=input_name)
    for N_conv,N_pooling,sz in zip(N_units['conv'], N_units['pooling'], kernel_size):
        try:
            L = layers.Conv1D(N_conv, sz, activation=None)(L)
        except:
            L = layers.Conv1D(N_conv, sz, activation=None)(inp)
        if activation_fun is not None:
            if activation_loc.lower() == 'after_conv':
                L = layers.ReLU()(L)
                L = layers.MaxPooling1D(N_pooling)(L)
            else:
                L = layers.MaxPooling1D(N_pooling)(L)
                L = layers.ReLU()(L)
        else:
            L = layers.MaxPooling1D(N_pooling)(L)
    return inp, L


def make_preprocessing_pipeline_2D(N_samples, N_units, kernel_size, activation_fun, activation_loc, input_name):
    if activation_fun is not None:
        if activation_fun.lower() not in ('relu',):
            raise Exception(f'Unknown activation function {activation_fun}')
        if activation_loc is None:
            raise Exception(f'Must specify activation function location')
        elif activation_loc.lower() not in ('after_conv', 'after_pooling'):
            raise Exception('activation_loc must be one of "after_conv" or "after_pooling"')
    inp = keras.Input(shape=(N_samples, 2, 1), name=input_name)
    for N_conv,N_pooling,sz in zip(N_units['conv'], N_units['pooling'], kernel_size):
        try:
            L = layers.Conv2D(N_conv, [sz, 2], padding='same', activation=None)(L)
        except:
            L = layers.Conv2D(N_conv, [sz, 2], padding='same', activation=None)(inp)
        if activation_fun is not None:
            if activation_loc.lower() == 'after_conv':
                L = layers.ReLU()(L)
                L = layers.MaxPooling2D([N_pooling, 1])(L)
            else:
                L = layers.MaxPooling2D([N_pooling, 1])(L)
                L = layers.ReLU()(L)
        else:
            L = layers.MaxPooling2D([N_pooling, 1])(L)
    return inp, L

# how many dimensions the input data should have
N_dims = 1
# whether to have a deeper network (2) or not (1)
depth_level = 1
# a dropout coefficient of 0 means no Dropout layer
dropout_coeff = 0
# learning rate of the Adam optimizer
learning_rate = 1e-4

N_units = {}

if depth_level == 1:
    N_units['conv'] = [16, 32, 64]
    N_units['dense'] = [64]
elif depth_level == 2:
    N_units['conv'] = [64, 128, 256, 512]
    N_units['dense'] = [128, 64]

N_units['pooling'] = [4 for _ in range(len(N_units['conv']))]
kernel_size = [5 for _ in range(len(N_units['conv']))]

if N_dims == 1:
    inputs = []
    L = []
    for var_name in var_names:
        inp,lyr = make_preprocessing_pipeline_1D(N_samples, N_units, kernel_size, \
                                                 'relu', 'after_conv', var_name)
        inputs.append(inp)
        L.append(lyr)
else:
    inputs,L = make_preprocessing_pipeline_2D(N_samples, N_units, kernel_size, \
                                              'relu', 'after_conv', '_'.join(var_names))

if isinstance(L, list):
    L = layers.concatenate(L)
L = layers.Flatten()(L)
for n in N_units['dense']:
    L = layers.Dense(n, activation='relu')(L)
if dropout_coeff > 0:
    L = layers.Dropout(dropout_coeff)(L)
output = layers.Dense(y['train'].shape[1])(L)

model = tf.keras.Model(inputs=inputs, outputs=output)

optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
loss = tf.keras.losses.MeanAbsoluteError()

model.compile(optimizer=optimizer, loss=loss)

model.summary()

In [ ]:
ts = strftime('%Y%m%d-%H%M%S', localtime())
path = 'experiments/' + ts
checkpoint_path = path + '/checkpoints'
os.makedirs(checkpoint_path)
keras.utils.plot_model(model, show_shapes=True, dpi=96)

#### Train the network

In [ ]:
if N_dims == 2:
    for key in x:
        x[key] = tf.transpose(x[key], perm=(1,2,0))

# Create a callback that saves the model's weights
cp_callback = keras.callbacks.ModelCheckpoint(filepath=checkpoint_path + \
                                                 '/weights.{epoch:02d}-{val_loss:.2f}.h5',
                                                 save_weights_only=False,
                                                 save_best_only=True,
                                                 monitor='val_loss',
                                                 verbose=0)

N_epochs = 5
batch_size = 128
N_batches = np.ceil(N_training_traces / batch_size)
steps_per_epoch = np.max([N_batches, 100])

if N_dims == 1:
    x_train = {name: x['train'][i] for i,name in enumerate(var_names)}
    x_validation = {name: x['validation'][i] for i,name in enumerate(var_names)}
else:
    x_train = x['train']
    x_validation = x['validation']

history = model.fit(x_train,
                    y['train'],
                    epochs = N_epochs,
                    batch_size = batch_size,
                    steps_per_epoch = steps_per_epoch,
                    validation_data = (x_validation, y['validation']),
                    verbose = 1,
                    callbacks = [cp_callback])

#### Plot the loss as a function of the epoch number

In [ ]:
epochs = np.r_[0 : len(history.history['loss'])] + 1
plt.plot(epochs, history.history['loss'], 'k', label='Training')
plt.plot(epochs, history.history['val_loss'], 'r', label='Validation')
plt.legend(loc='best')
plt.xlabel('Epoch')
plt.ylabel('Loss');

#### Find the best model based on the validation loss

In [ ]:
checkpoint_files = glob.glob(checkpoint_path + '/*.h5')
val_loss = [float(file[:-3].split('-')[-1]) for file in checkpoint_files]
best_checkpoint = checkpoint_files[np.argmin(val_loss)]
best_model = tf.keras.models.load_model(best_checkpoint)

#### Compute the network prediction on the test set

In [ ]:
if N_dims == 1:
    y_prediction = best_model.predict({name: x['test'][i] for i,name in enumerate(var_names)})
else:
    y_prediction = best_model.predict(x['test'])

#### Compute the Mean Absolute Percentage Error on the CNN prediction

In [ ]:
y_test = np.squeeze(y['test'].numpy())
mape_prediction = tf.keras.losses.mean_absolute_percentage_error(y_test.T, y_prediction.T).numpy()
for generator_ID, mape in zip(generator_IDs, mape_prediction):
    print(f'MAPE on CNN prediction for generator {generator_ID} ... {mape:.2f}%')

#### Plot the results obtained with the CNN

In [ ]:
block_size = y['test'].shape[0] // n_generators
fig,ax = plt.subplots(1, n_generators, figsize=(4 * n_generators,4))
if n_generators == 1:
    ax = [ax]
y_max = np.max(y['train'], axis=0)
y_min = np.min(y['train'], axis=0)
for i in range(n_generators):
    limits = [y_min[i], y_max[i]+1]
    ax[i].plot(limits, limits, 'g--')
    idx = np.arange(i * block_size, (i+1) * block_size)
    ax[i].plot(y['test'][i * block_size : (i+1) * block_size, i], \
               y_prediction[i * block_size : (i+1) * block_size, i], 'o', \
               color=[1,.7,1], markersize=4, markerfacecolor='w', markeredgewidth=1)
    for j in range(int(limits[0]), int(limits[1])):
        idx, = np.where(np.abs(y['test'][i * block_size : (i+1) * block_size, i] - (j + 1/3)) < 1e-3)
        m = np.mean(y_prediction[idx + i * block_size,i])
        s = np.std(y_prediction[idx + i * block_size,i])
        ax[i].plot(j+1/3 + np.zeros(2), m + s * np.array([-1,1]), 'm-', linewidth=2)
        ax[i].plot(j+1/3, m, 'ms', markersize=8, markerfacecolor='w', \
                markeredgewidth=2)
    ax[i].axis([1.8, limits[1], 1.8, limits[1]])
    ax[i].set_xlabel('Expected value')
    ax[i].set_title(f'Generator {generator_IDs[i]}')
ax[0].set_ylabel('Predicted value')
fig.tight_layout()